In [1]:
from scapy.all import *
load_contrib('bgp') #scapy does not automatically load items from Contrib. Must call function and module name to load.


class MutablePacket():
    def __init__(self, pkt):
        self.pkt = pkt
        self.been_modified = False
        self.bgp_updates = []

    def is_bgp(self):
        if (str(self.summary()).find('BGPHeader') > 0):
            return True
        return False
    
    def is_bgp_update(self):
        if (self.is_bgp() and self.pkt[BGPHeader].type == 2):
            return True
        return False

    def has_nlri_advertisements(self):
        if self.pkt[BGPUpdate].path_attr_len > 0:
            return True
        return False
    
    def has_withdraw_routes(self):
        if self.pkt[BGPUpdate].withdrawn_routes_len > 0:
            return True
        return False

    def add_bgp_update(self, payload):
        self.bgp_updates.append(payload)


    def get_segment_length(self):
        return self.pkt[BGPUpdate].path_attr[1].attribute.segments[0].segment_length

    def get_segment_asn(self):
        return self.pkt[BGPUpdate].path_attr[1].attribute.segments[1].segment_length

    def get_bgp_segment_next_hop(self):
        return self.pkt[BGPUpdate].path_attr[2].attribute.next_hop

    def print_bgp_update_summary(self):
        print ("    Destination IP = " + self.pkt[IP].dst) #Local AS
        print ("    Source IP = " + self.pkt[IP].src) #Remote AS
        print ("    BGP Segment AS = " + str(self.get_segment_asn())) #even though it says segment length, that field is used to announce the A>
        print ("    BGP Segment Next Hop = " + str(self.get_bgp_segment_next_hop()))

    def get_nlris(self):
        return self.pkt[BGPUpdate].nlri

    def get_adv_segment(self, nlri):
        return [self.get_segment_asn(), str(nlri.prefix).split('/')[0], str(nlri.prefix).split('/')[1]]

    def bytes(self):
        return bytes(self.pkt)

    def byte_array(self):
        return bytearray(self.pkt.bytes())

    def remove_nlri(self, nlri, update):
        bgp_update = self.pkt.getlayer(scapy.contrib.bgp.BGPUpdate, update.get_layer_index())

        nlri_bytes = bytes(nlri)
        print("len of nlri: " + str(len(nlri_bytes)))
        # pkt_byte_array = bytearray(bytes(self.pkt))
        update_byte_array = bytearray(bytes(bgp_update))
        
        try: 
            # get index of nlri in the update
            index = update_byte_array.index(nlri_bytes)
            print("start index of nlri to remove: " + str(index))

            # delete the nlri from the update
            del update_byte_array[index:index+len(nlri_bytes)]

            # convert self.pkt to bytearray
            pkt_bytes = bytearray(bytes(self.pkt))
        
            # get index of update in the packet
            pkt_index = pkt_bytes.index(bytes(bgp_update))
            print(pkt_index)
            
            # delete the entire update from the packet
            del pkt_bytes[pkt_index:pkt_index+len(bytes(bgp_update))]

            # replace add in the modified update where the old update was
            pkt_bytes[pkt_index:pkt_index] = update_byte_array
            
            # convert the modified bytearray back to a packet
            self.pkt = IP(bytes(pkt_bytes))
            
            # update the bgp header length
            bgp_header = self.pkt.getlayer(scapy.contrib.bgp.BGPHeader, update.get_layer_index())
            bgp_header.len = bgp_header.len - len(nlri_bytes)

            # update pkt checksums, and lengths, set modified flag
            del self.pkt[IP].chksum
            del self.pkt[TCP].chksum
            self.pkt[IP].len = self.pkt[IP].len - len(nlri_bytes)
            print("modified packet: ") 
            self.pkt.show2()
            self.set_modified()
        except ValueError as v:
            print("Error. nlri not found in packet. this is weird: " + repr(v))
            print("nlri not found:")
            nlri.show()

    def is_modified(self):
        return self.been_modified

    def set_modified(self):
        self.been_modified = True

    def packet(self):
        return self.pkt

    def summary(self):
        return self.pkt.summary()

    def show(self):
        return self.pkt.show()

    def iterpayloads(self):
        return self.pkt.iterpayloads()

    def iterate_over_payload_nlris(self, most_recent_bgp_header, payload):
        print("iterate over payload nlris")
        if not self.has_withdraw_routes() and self.has_nlri_advertisements():
            asn = get_origin_asn_from_payload(payload)
            print('ASN is: '+ str(asn) + ' Checking prefixes:....')
            print("origin asn: " + str(payload.path_attr[1].attribute.show()))

            for num, item in enumerate(payload.nlri):
                segment = [asn, payload.nlri[num].prefix.split('/')[0], payload.nlri[num].prefix.split('/')[1]]
                print(segment)




In [21]:
class BGPUpdate2:
    def __init__(self, bgp_header, bgp_update, layer_index):
        self.bgp_header = bgp_header
        self.bgp_update = bgp_update
        self.layer_index = layer_index
        self.origin_asn = -1
        self.next_hop = None

    def get_layer_index(self):
        return self.layer_index

    def get_origin_asn(self):
        return self.origin_asn
    
    def has_nlri_advertisements(self):
        if self.bgp_update.path_attr_len > 0:
            return True
        return False
    
    def has_withdraw_routes(self):
        if self.bgp_update.withdrawn_routes_len > 0:
            return True
        return False

    
    def get_origin_asn_from_payload(self):
        if len(self.bgp_update.path_attr[1].attribute.segments[-1].segment_value) > 1: #grabs last asn in as_path (origin)
            self.origin_asn = self.bgp_update.path_attr[1].attribute.segments[-1].segment_value[-1]
        else:
            print('reverting to segment length')
            self.origin_asn = self.bgp_update.path_attr[1].attribute.segments[-1].segment_length
        return self.origin_asn

    def nlri(self):
        return self.bgp_update.nlri
   
    def get_segment(self, nlri):
        if self.origin_asn == -1:
            self.get_origin_asn_from_payload()
        return [self.origin_asn, str(nlri.prefix).split('/')[0], str(nlri.prefix).split('/')[1]]
    
    def get_next_hop_asn(self):
        for i in self.bgp_update.path_attr:
            if i.type_code == 0x03: # 0x02 == AS_PATH
#                 print(i.attribute.show())
#                 print(i.attribute.next_hop)
                self.next_hop = i.attribute.next_hop
                print("Next Hop ASN: " + str(self.next_hop))

In [3]:
from enum import Enum
class ValidatePrefixResult2(Enum):
    prefixValid = 0
    prefixNotRegistered = 1
    prefixOwnersDoNotMatch = 2
    
#Chain check function. Needs to be updated with smart contract calls.  
def bgpchain_validate(segment, tx_sender):
    print ("Validating segment.....")
#     print (tx_sender)
    inIP = IPv4Address(segment[1])
    print (inIP)
    inSubnet = int(segment[2])
    print (str(inSubnet))
    inASN = int(segment[0])
    print (str(inASN))

    print ("Checking segment: AS" + str(inASN)+ " , " + str(inIP) + "/" + str(inSubnet))
    # [11872, '128.230.64.0', '18']

    if str(inASN) == '11872' and str(inIP) == '128.230.64.0' and str(inSubnet) == '18':
        validationResult = 2
    elif str(inASN) == '11' and str(inIP) == '10.11.0.0' and str(inSubnet) == '24':
        validationResult = 1
    elif str(inASN) == '4' and str(inIP) == '10.4.0.0' and str(inSubnet) == '24':
        # [4, '10.4.0.0', '24']
        validationResult = 1
    else:
        validationResult = 0

#     validationResult = tx_sender.tx.sc_validatePrefix(int(inIP), inSubnet, inASN)
    print(str(validationResult))
    return validationResult

In [4]:
from scapy.all import *
load_contrib('bgp')
from ipaddress import IPv4Address


In [9]:
base = "/home/greg/BGP_Chain/Seed_scalable_complete/bgp_smart_contracts/pcaps/"
# base = "/Users/gcusack/Downloads/"
pcap_path = base + "multiple-updates.pcap"
# pcap_path = base + "update.pcap"

In [10]:
def get_bgp_update_packet():
    for pkt in PcapReader(pcap_path):
        try:
            if (str(pkt.summary()).find('BGPHeader') > 0) and (pkt[BGPHeader].type == 2):
                return pkt
                print(pkt[TCP].show())
    #             print(packet[TCP].payload())
        except Exception as e:
            print(repr(e))

In [11]:
pkt = get_bgp_update_packet()
pkt = pkt[IP]
m_pkt = MutablePacket(pkt)

In [12]:
m_pkt.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0xc0
  len       = 641
  id        = 4998
  flags     = DF
  frag      = 0
  ttl       = 1
  proto     = tcp
  chksum    = 0x4db2
  src       = 10.104.0.12
  dst       = 10.104.0.164
  \options   \
###[ TCP ]### 
     sport     = bgp
     dport     = 59473
     seq       = 2063855751
     ack       = 61297140
     dataofs   = 8
     reserved  = 0
     flags     = PA
     window    = 510
     chksum    = 0x17f3
     urgptr    = 0
     options   = [('NOP', None), ('NOP', None), ('Timestamp', (3262136288, 1326917326))]
###[ HEADER ]### 
        marker    = 0xffffffffffffffffffffffffffffffff
        len       = 152
        type      = UPDATE
###[ UPDATE ]### 
           withdrawn_routes_len= 0
           \withdrawn_routes\
           path_attr_len= 83
           \path_attr \
            |###[ BGPPathAttr ]### 
            |  type_flags= Transitive
            |  type_code = ORIGIN
            |  attr_len  = 1
            |  \attri

In [13]:
class validatePrefixResult2(Enum):
    prefixValid = 0
    prefixNotRegistered = 1
    prefixOwnersDoNotMatch = 2


In [14]:
def handle_invalid_advertisement(m_pkt, nlri, validationResult, update):
    print ("AS " + str(update.get_origin_asn()) + " Failed Authorization. [" + str(validationResult) + "]. BGPUpdate layer: " + str(update.get_layer_index()))
    print("modifying packet: ")
    remove_invalid_nlri_from_packet(m_pkt, nlri, update)


def remove_invalid_nlri_from_packet(m_pkt, nlri, update):
    m_pkt.remove_nlri(nlri, update)
    if m_pkt.is_modified():
        print("packet modified")
    else:
        print("ERROR: packet modification failed")

In [25]:

tx_sender = "test..."
if m_pkt.is_bgp_update(): # checks for both bgp packet and bgp update
    print("rx BGP Update pkt")
    try:
        # iterate over packet bgp payloads (bgp layers)
        layer_index = 0
        for payload in m_pkt.iterpayloads():
            if isinstance(payload,  scapy.contrib.bgp.BGPHeader):
                most_recent_bgp_header = payload
            elif isinstance(payload, scapy.contrib.bgp.BGPUpdate):
                layer_index += 1
                print(type(payload))
                # m_pkt.add_bgp_update(BGPUpdate())
                update = BGPUpdate2(most_recent_bgp_header, payload, layer_index)
                if not update.has_withdraw_routes() and update.has_nlri_advertisements():
                    update.get_next_hop_asn()
                    # origin_asn = get_update_origin_asn(update)
                    for count, nlri in enumerate(update.nlri()):
#                         update.get_next_hop_asn()
                        segment = update.get_segment(nlri)
                        print("nlri count: " + str(count))
                        print("BGP NLRI check: " + str(nlri.prefix))
                        print ("Advertised Segment: " + str(segment))
                        print ("validating advertisement for ASN: " + str(update.get_origin_asn()))

                        validationResult = bgpchain_validate(segment, tx_sender)
                        if validationResult == 0:
                            print("NLRI " + str(count) + " passed authorization...checking next ASN")
                        elif validationResult == 1:
                            handle_invalid_advertisement(m_pkt, nlri, validationResult, update)
                        elif validationResult == 2:
                            print("prefix owners do not match")
                            handle_invalid_advertisement(m_pkt, nlri, validationResult, update)
                        else:
                            print("error. should never get here. received back unknown validationResult: " + str(validationResult))
                    if m_pkt.is_modified():
                        print("BGP Update packet has been modified")
                else:
                    print("BGP Update packet has no NLRI advertisements")
            else:
                print("Packet layer is not a BGPUpdate or BGPHeader layer")

        print ("All Advertised ASN's within all BGP Updates have been checked")
        if m_pkt.is_modified():
            print("setting modified packet payload")
#             packet.set_payload(m_pkt.bytes())
        else:
            print("packet not modified. accepting as is")
#         packet.accept()

    except IndexError as ie:
        print("index error. diff type of bgp announcement. accept packet. error: " + repr(ie))
#         packet.accept()
        print("accepted other bgp type packet")
    except Exception as e: 
        print("bgp msg other: " + repr(e))
        # packet.accept()
        pass
else:
    print("not a bgp update packet. accept packet")
#     packet.accept()

rx BGP Update pkt
Packet layer is not a BGPUpdate or BGPHeader layer
Packet layer is not a BGPUpdate or BGPHeader layer
<class 'scapy.contrib.bgp.BGPUpdate'>
Next Hop ASN: 10.104.0.12
nlri count: 0
BGP NLRI check: 128.230.192.0/18
Advertised Segment: [11872, '128.230.192.0', '18']
validating advertisement for ASN: 11872
Validating segment.....
128.230.192.0
18
11872
Checking segment: AS11872 , 128.230.192.0/18
0
NLRI 0 passed authorization...checking next ASN
nlri count: 1
BGP NLRI check: 149.119.128.0/18
Advertised Segment: [11872, '149.119.128.0', '18']
validating advertisement for ASN: 11872
Validating segment.....
149.119.128.0
18
11872
Checking segment: AS11872 , 149.119.128.0/18
0
NLRI 1 passed authorization...checking next ASN
nlri count: 2
BGP NLRI check: 149.119.64.0/18
Advertised Segment: [11872, '149.119.64.0', '18']
validating advertisement for ASN: 11872
Validating segment.....
149.119.64.0
18
11872
Checking segment: AS11872 , 149.119.64.0/18
0
NLRI 2 passed authorization.

In [67]:
import psutil
import socket
import fcntl
import struct
import netifaces as ni



def get_ip_address(ifname):
#     ip = ni.ifaddresses(ifname)[ni.AF_INET][0]['addr']
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    print(s)
    return socket.inet_ntoa(fcntl.ioctl(
        s.fileno(),
        0x8915,  # SIOCGIFADDR
        struct.pack('256s', bytes(ifname[:15], 'utf-8'))
#         s.pack('256s', ifname[:15])
    )[20:24])

def get_interface_names():
    addrs = psutil.net_if_addrs()
    print(addrs.keys())
    return addrs.keys()

def get_interface_ips(interface_names):
    ips = []
    for interface_name in interface_names:
        print("interface_name: " + interface_name)
        if interface_name == "lo":
            continue
        ip_addr = get_ip_address(interface_name)
        print(ip_addr)
        ips.append(ip_addr)
    return ips

In [41]:
def get_flow_direction():
    interface_names = get_interface_names()
    interface_ips = get_interface_ips(interface_names)
#     print(interface_ips)
#     if src_ip in interface_ips:
#         return FlowDirection.outbound
#     else:
#         return FlowDirection.inbound

In [52]:
get_flow_direction()

dict_keys(['lo', 'wlp1s0', 'docker0', 'br-6d0a0accef21', 'br-fa4f0e1c9871', 'br-1c2031f9760e', 'br-c210aa28d451', 'br-a07928133c6e', 'br-32a5410d3140', 'br-355c76179464', 'br-36de9210dc47', 'br-5a78b2ad0735', 'br-d91ca3e074be', 'br-407a1ecc9e36', 'br-a0b753e2b75a', 'veth6f0279d', 'vethad9c72b', 'veth8fcc931', 'veth9f26863', 'vetha8079ce', 'veth97cc001', 'veth6756fe1', 'vethb3a209f', 'veth83f8985', 'veth2d96d35', 'veth92f4eae', 'enp0s31f6'])
interface_name: lo
interface_name: wlp1s0
<socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_DGRAM, proto=0, laddr=('0.0.0.0', 0)>


error: argument for 's' must be a bytes object

In [68]:
get_ip_address('lo')
# type(0x123)

<socket.socket fd=61, family=AddressFamily.AF_INET, type=SocketKind.SOCK_DGRAM, proto=0, laddr=('0.0.0.0', 0)>


'127.0.0.1'